# Creating a Mosaic from OPERA RTC-S1 Images

In this notebook, we will work with products from the Observational Products for End-Users from Remote Sensing Analysis (OPERA) project at the Jet Propolsion Labratory (JPL). OPERA data include Radiometric Terrain Correction (RTC) Synthetic Aperature Radar (SAR) images, which remove distortions by referencing the Copernicus global 30-m (GLO-30) Digital Elevation Model (DEM).

OPERA Products can be accessed using the Alaska Satellite Facility (ASF) SAR Data Search tools via the graphical interface [Vertex](https://search.asf.alaska.edu/#/), the [ASF API](https://docs.asf.alaska.edu/api/basics/) or using the  [`asf_search`](https://docs.asf.alaska.edu/asf_search/basics/) python module. If you do not already have one, please create a free [Earthdata Login Account](https://urs.earthdata.nasa.gov/users/new) to access products using ASF Search Tools.

In this module, we will access OPERA RTC images using the `asf_search` python module, download the selected bursts, and merge them to create a mosaic.

## 1. Enter Earthdata Login credentials
Earthdata credentials are required to access OPERA datasets. One way do this is to store your username and password in a `netrc` file. 

In [ ]:
import os
from getpass import getpass

os.environ['GDAL_HTTP_COOKIEFILE'] = '/tmp/cookies.txt'
os.environ['GDAL_HTTP_COOKIEJAR'] = '/tmp/cookies.txt'
os.environ['GDAL_HTTP_NETRC'] = 'YES'
os.environ['GDAL_HTTP_NETRC_FILE'] = 'temp_netrc'

username = getpass('NASA Earthdata Login username: ')
password = getpass('NASA Earthdata Login password: ')

with open(os.environ['GDAL_HTTP_NETRC_FILE'], 'w') as f:
    f.write(f'machine urs.earthdata.nasa.gov login {username} password {password}')

## 2. Search for OPERA RTC products over San Francisco
After checking we have our environment correctly set up, we can perform the search for our desired products.The polygon in the search is centered over San Francisco.

In [ ]:
import asf_search

options = {
	'intersectsWith': 'POLYGON((-122.5306 37.6278,-122.1372 37.6207,-122.1372 37.9566,-122.5287 37.9554,-122.5306 37.6278))',
	'dataset': 'OPERA-S1',
	'start': '2023-11-14T14:00:00Z',
	'end': '2023-11-14T15:00:00Z',
	'flightDirection': 'DESCENDING',
	'processingLevel': [
		'RTC'
	],
	'maxResults': '1000'
}

results = asf_search.search(**options)
print(f'Found {len(results)} products that match your search criteria')

## 3. Get URLs for selected bursts from search results

In [ ]:
polarization = 'VV'

urls = []
for result in results:
	for url in result.umm['RelatedUrls']:
		if url['URL'].startswith('https://') and url['URL'].endswith(f'{polarization}.tif'):
			urls.append('/vsicurl/' + url['URL'])

## 4. Merge bursts into a single tif and save

In [ ]:
from osgeo import gdal
import osgeo_utils.gdal_merge as gdal_merge

gdal.DontUseExceptions()
output_file_path = f'{os.getcwd()}/OPERA_RTC_San_Francisco.tif'

parameters = ['', '-a_nodata', 'nan', '-o', output_file_path] + urls
gdal_merge.main(parameters)

## 5. Plot merged GeoTIFF

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import rasterio
import numpy as np

src = rasterio.open(output_file_path)
arr = src.read(1)
masked_array = np.ma.array(arr, mask=(arr==0.0))

f, ax = plt.subplots(1,1, figsize=(10,10))

im = ax.imshow(masked_array, cmap='grey', vmin=0, vmax=0.25)
ax.set_title('Merged OPERA RTC VV Products', fontsize=16)
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

cbar = plt.colorbar(im, shrink=0.6, ticks=[0, 0.1, 0.2])
cbar.set_label('Backscatter (Power)', fontsize=12)
cbar.ax.set_yticklabels(['0', '0.1', '0.2'])
cbar.ax.yaxis.set_label_position('left')

plt.tight_layout()